
# DRY & Code Reusability — Notebook  
**Don’t Repeat Yourself (DRY) & Reużywalność kodu — notatnik szkoleniowy**

**Audience / Grupa docelowa:** Python intermediate  
**Time / Czas:** ~45–60 min  
**Format:** Bilingual (EN/PL) — dwujęzycznie



## Learning objectives / Cele nauki

**EN**  
By the end of this notebook, you will be able to:  
- Explain the **DRY** principle and identify duplication smells.  
- Refactor duplication into **functions**, **parameters**, and **modules**.  
- Apply **composition**, **higher-order functions**, and **decorators** to reuse code.  
- Structure code for reuse across files and projects.  
- Write exercises that reinforce DRY thinking.

**PL**  
Po zakończeniu tego notatnika będziesz umieć:  
- Wyjaśnić zasadę **DRY** i rozpoznawać symptomy duplikacji.  
- Refaktoryzować duplikacje do **funkcji**, **parametrów** i **modułów**.  
- Stosować **kompozycję**, **funkcje wyższego rzędu** oraz **dekoratory** do ponownego użycia kodu.  
- Strukturyzować kod tak, aby był wielokrotnie wykorzystywany w różnych plikach i projektach.  
- Rozwiązywać ćwiczenia utrwalające myślenie zgodne z DRY.



## What is DRY? / Czym jest DRY?

**EN**  
**DRY = Don't Repeat Yourself.** Every piece of knowledge should have a **single, unambiguous, authoritative** representation in the system. Duplication increases maintenance cost and the risk of inconsistencies.

**Common duplication smells:**  
- Copy–paste blocks that differ in **one or two lines**.  
- The same constant/business rule scattered across files.  
- Repeated **I/O patterns** (open → transform → save).  
- Parallel functions that only differ by a **parameter**.

**PL**  
**DRY = Nie powtarzaj się.** Każda porcja wiedzy w systemie powinna mieć **jedno, jednoznaczne, autorytatywne** źródło. Duplikacja podnosi koszt utrzymania i ryzyko niespójności.

**Typowe sygnały duplikacji:**  
- Bloki „kopiuj‑wklej” różniące się **jedną–dwiema liniami**.  
- Ta sama stała/reguła biznesowa rozrzucona po plikach.  
- Powtarzalne schematy **wejścia/wyjścia** (open → transform → save).  
- Bliźniacze funkcje różniące się tylko **parametrem**.



## Refactor duplication into a function / Refaktoryzacja do funkcji

**EN**: Start by extracting the shared steps into a named function.  
**PL**: Zacznij od wyodrębnienia wspólnych kroków do funkcji nazwanej.


In [ ]:

# BEFORE (duplication)
def price_with_vat_pl(price):
    return round(price * 1.23, 2)

def price_with_vat_de(price):
    return round(price * 1.19, 2)

# AFTER (parameterize differences)
def price_with_vat(price, vat_rate):
    return round(price * (1 + vat_rate), 2)

print(price_with_vat(100, 0.23))
print(price_with_vat(100, 0.19))



## Parameterization over duplication / Parametryzacja zamiast duplikacji

**EN**  
If two functions only differ in **constants** or **small behaviors**, parameterize them.  
**PL**  
Jeśli dwie funkcje różnią się tylko **stałymi** lub **drobiazgami**, zastosuj parametry.


In [ ]:

from pathlib import Path

def transform_lines(src: Path, dst: Path, *, to_upper: bool = False, prefix: str = ""):
    """Generic file transformer—reusable.
    - Set to_upper to transform case
    - Use prefix to add annotation per line
    """
    with src.open(encoding="utf-8") as fin, dst.open("w", encoding="utf-8") as fout:
        for line in fin:
            line = line.rstrip("\n")
            if to_upper:
                line = line.upper()
            if prefix:
                line = f"{prefix}{line}"
            print(line, file=fout)



## Reuse via composition / Reużywalność przez kompozycję

**EN**  
Prefer **composition** (build from smaller parts) over deep inheritance hierarchies.  
**PL**  
Preferuj **kompozycję** (składanie z mniejszych klocków) zamiast głębokiej hierarchii dziedziczenia.


In [ ]:

class Formatter:
    def __init__(self, prefix=""):
        self.prefix = prefix
    def format(self, text: str) -> str:
        return f"{self.prefix}{text}"

class Writer:
    def write(self, text: str):
        print(text)

class Service:
    def __init__(self, formatter: Formatter, writer: Writer):
        self.formatter = formatter
        self.writer = writer
    def process(self, text: str):
        self.writer.write(self.formatter.format(text))

svc = Service(Formatter(prefix="[INFO] "), Writer())
svc.process("Hello")



## Higher‑order functions / Funkcje wyższego rzędu

**EN**  
Pass behavior as a function instead of forking code.  
**PL**  
Przekazuj zachowanie jako funkcję zamiast rozgałęziać kod.


In [ ]:

from typing import Callable, Iterable, TypeVar, List

T = TypeVar("T")
U = TypeVar("U")

def map_and_filter(items: Iterable[T], map_fn: Callable[[T], U], pred: Callable[[U], bool]) -> List[U]:
    return [y for x in items if (y := map_fn(x)) and pred(y)]

data = ["  Alice ", "bob", "  ", "Eve "]
result = map_and_filter(data, lambda s: s.strip().title(), lambda s: len(s) >= 3)
print(result)  # ['Alice', 'Bob', 'Eve']



## Decorators for cross‑cutting concerns / Dekoratory dla aspektów przekrojowych

**EN**  
Use decorators to **reuse** logging, timing, caching across many functions.  
**PL**  
Użyj dekoratorów do wielokrotnego użytku logowania, pomiaru czasu, cache’owania.


In [ ]:

import time
from functools import wraps

def timed(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        try:
            return fn(*args, **kwargs)
        finally:
            dur = (time.perf_counter() - start) * 1000
            print(f"{fn.__name__} took {dur:.2f} ms")
    return wrapper

@timed
def slow_sum(n: int) -> int:
    total = 0
    for i in range(n):
        total += i
    return total

slow_sum(200000)



## Structure for reuse: modules & packages / Struktura do reużywalności: moduły i pakiety

**EN**  
- Extract utilities into **modules** (e.g., `utils/io.py`, `utils/text.py`).  
- Keep public API small and documented (`__all__`).  
- Add **type hints** to improve IDE support and safe reuse.  
- Write **unit tests** to lock behavior and allow safe refactoring.

**PL**  
- Wyodrębniaj narzędzia do **modułów** (np. `utils/io.py`, `utils/text.py`).  
- Utrzymuj małe i opisane **publiczne API** (`__all__`).  
- Dodawaj **adnotacje typów** dla wsparcia IDE i bezpiecznej reużywalności.  
- Pisz **testy jednostkowe**, aby zabezpieczyć zachowanie i umożliwić refaktoryzacje.



## Example: Reusable data pipeline function / Przykład: Reużywalna funkcja „pipeline”


In [ ]:

from typing import Iterable, Callable, TypeVar

T = TypeVar("T")

def pipeline(data: Iterable[T], *steps: Callable[[Iterable[T]], Iterable[T]]) -> Iterable[T]:
    for step in steps:
        data = step(data)
    return data

def drop_empty(xs: Iterable[str]) -> Iterable[str]:
    return (x for x in xs if x.strip())

def to_int(xs: Iterable[str]) -> Iterable[int]:
    for x in xs:
        try:
            yield int(x)
        except ValueError:
            continue

def keep_even(xs: Iterable[int]) -> Iterable[int]:
    return (x for x in xs if x % 2 == 0)

data = ["10", "  ", "3", "6", "foo", "8"]
res = list(pipeline(data, drop_empty, to_int, keep_even))
print(res)  # [10, 6, 8]



## Anti‑patterns / Antywzorce

**EN**  
- Copy–paste then tweak.  
- Monster functions with optional flags controlling many behaviors.  
- Over‑engineering (abstract too early).  
- DRYing **comments** instead of **code**.

**PL**  
- Kopiuj‑wklej, a potem „dokręcanie śrubek”.  
- „Potworne” funkcje z dziesiątkami flag.  
- Nadmierna abstrakcja zbyt wcześnie.  
- „Wysuszanie” **komentarzy** zamiast **kodu**.



# Exercises / Ćwiczenia

**Guidelines (EN):** Prefer parameterization, composition, or higher‑order functions over duplication.  
**Wskazówki (PL):** Wybieraj parametryzację, kompozycję lub funkcje wyższego rzędu zamiast duplikacji.



## Exercise 1 — Parameterize VAT / Parametryzacja VAT

**EN**  
You have two functions: `invoice_total_pl(items)` and `invoice_total_de(items)` that differ only in VAT rate.  
Refactor into a single reusable function with a `vat_rate` parameter.

**PL**  
Masz dwie funkcje: `invoice_total_pl(items)` i `invoice_total_de(items)` różniące się jedynie stawką VAT.  
Zrefaktoryzuj do jednej funkcji z parametrem `vat_rate`.


In [ ]:

# STARTER
items = [100.0, 49.9, 250.0]

def invoice_total_pl(items):
    return round(sum(items) * 1.23, 2)

def invoice_total_de(items):
    return round(sum(items) * 1.19, 2)

# TODO: write a single function: invoice_total(items, vat_rate)
# and show results for PL and DE



## Exercise 2 — Remove copy‑paste I/O / Usuń duplikację I/O

**EN**  
Two functions `save_upper(src, dst)` and `save_lower(src, dst)` only differ in case transformation.  
Create **one** function that accepts a transformation callable.

**PL**  
Dwie funkcje `save_upper(src, dst)` oraz `save_lower(src, dst)` różnią się tylko zmianą wielkości liter.  
Utwórz **jedną** funkcję przyjmującą funkcję transformującą.


In [ ]:

# STARTER
def save_upper(text: str, path: str):
    with open(path, "w", encoding="utf-8") as f:
        f.write(text.upper())

def save_lower(text: str, path: str):
    with open(path, "w", encoding="utf-8") as f:
        f.write(text.lower())

# TODO: create save_transformed(text, path, transform)
# Example usage:
# save_transformed("Hello", "out.txt", str.title)



## Exercise 3 — Decorator reuse / Reużywalność z dekoratorem

**EN**  
Write a decorator `cached_once` that memoizes a **no‑argument** function the first time it is called.  
Then apply it to two independent functions to avoid repeated heavy work.

**PL**  
Napisz dekorator `cached_once`, który zapamiętuje wynik funkcji **bezargumentowej** przy pierwszym wywołaniu.  
Zastosuj go do dwóch niezależnych funkcji, aby uniknąć kosztownych obliczeń.


In [ ]:

# STARTER
import time

def heavy():
    time.sleep(0.2)
    return "OK"

# TODO:
# @cached_once
# def config():
#     return heavy()
#
# @cached_once
# def exchange_rate():
#     return heavy()



# Solutions / Rozwiązania

*(Try first before scrolling here / Najpierw spróbuj samodzielnie!)*


In [ ]:

# Solution 1
def invoice_total(items, vat_rate):
    return round(sum(items) * (1 + vat_rate), 2)

print("PL:", invoice_total([100.0, 49.9, 250.0], 0.23))
print("DE:", invoice_total([100.0, 49.9, 250.0], 0.19))


In [ ]:

# Solution 2
from typing import Callable

def save_transformed(text: str, path: str, transform: Callable[[str], str] = lambda s: s):
    with open(path, "w", encoding="utf-8") as f:
        f.write(transform(text))

# demo (writes a file 'demo.txt')
save_transformed("hello WORLD", "demo.txt", str.title)
with open("demo.txt", encoding="utf-8") as f:
    print(f.read())


In [ ]:

# Solution 3
from functools import wraps

def cached_once(fn):
    has = {"done": False, "value": None}
    @wraps(fn)
    def wrapper():
        if not has["done"]:
            has["value"] = fn()
            has["done"] = True
        return has["value"]
    return wrapper

import time

def heavy():
    time.sleep(0.2)
    return "OK"

@cached_once
def config():
    return heavy()

@cached_once
def exchange_rate():
    return heavy()

t0 = time.perf_counter()
print(config(), exchange_rate())
t1 = time.perf_counter()
print(f"Total time ~{(t1 - t0)*1000:.1f} ms (should be ~200 ms, not ~400 ms)")



## References / Źródła

- **Python docs — Functions / Funkcje:** https://docs.python.org/3/tutorial/controlflow.html#defining-functions  
- **Python docs — Modules / Moduły:** https://docs.python.org/3/tutorial/modules.html  
- **Typing — type hints / adnotacje:** https://docs.python.org/3/library/typing.html  
- **functools (LRU cache, wraps):** https://docs.python.org/3/library/functools.html  
- **itertools (combinators):** https://docs.python.org/3/library/itertools.html  
- **unittest — testy jednostkowe:** https://docs.python.org/3/library/unittest.html  
- **Refactoring (Martin Fowler):** https://martinfowler.com/books/refactoring.html
